In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch



torch.cuda.empty_cache()

# 设置设备为 GPU，如果有可用 GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载 CodeLlama 模型和分词器
# model_name = "codellama/CodeLlama-13b-Instruct-hf"
# model_name = "Qwen/Qwen2.5-1.5B-Instruct"
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [ ]:
# !pip install openai


import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
# 目标目录，包含所有文件夹和 Java 文件
# directory_path = '/content/drive/MyDrive/Bloodsugar_reversed/src/main/java'
directory_path = '/content/drive/MyDrive/zitmo_banker_reversed/src/main/java/com/android/security'

# 输出目录，用于保存生成的摘要文件
output_directory = '/content/drive/MyDrive/zitmo_banker_reversed/summaries'
os.makedirs(output_directory, exist_ok=True)  # 确保输出目录存在

# 使用 os.walk() 遍历目标目录下的所有子目录和文件
for root, dirs, files in os.walk(directory_path):
    for filename in files:
        if filename.endswith(".java"):
            # 构建完整的文件路径
            torch.cuda.empty_cache()
            file_path = os.path.join(root, filename)

            # 读取 Java 文件的内容
            with open(file_path, 'r', encoding='utf-8') as java_file:
                java_code = java_file.read()

            # 构建输入格式，添加注释提示
            input_text =f"""
            Please analyze the following Java code file and generate a concise code summary that describes the main functionality, operations, and behaviors.
            Focus on the purpose of each method, key variables, and any important conditional logic.
            Ensure the summary is clear and accurate, suitable for storage in a vector database for later use in Retrieval-Augmented Generation (RAG).
            Here’s the code:
            \n\n{java_code}




            """
            #             The generated summary should include:

            # Primary functionality and purpose of the code: Briefly explain the core operation of the code.
            # Key operations and behaviors: Describe the main workflow at the file or method level.
            # Important methods and their roles: Outline the purpose of each main method.
            # Security and sensitive operations: If the code involves permissions, data storage, network access, or other sensitive actions, highlight these specifically.
            # If the code is lengthy, consider summarizing each method individually and then combining them into a final summary.
            # Tokenize 输入
            inputs = tokenizer(input_text, return_tensors="pt").to(device)
            input_ids = tokenizer.encode(input_text, return_tensors='pt')
            attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=device)

            # 生成摘要（这里的 max_length 可以根据需要调整）
            outputs = model.generate(
                inputs.input_ids,
                attention_mask=attention_mask,
                pad_token_id=tokenizer.eos_token_id,
                max_new_tokens=500,  # 设置你希望生成的新 token 数量
                do_sample=True
            )

            # 解码输出，生成代码摘要
            summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # 提取 "Code Summary below:" 后面的内容
            summary_content = summary.split("Code Summary below:")[-1].strip()
            # print(summary_content)

            # 输出摘要文件的路径
            # 创建与文件所在目录结构相对应的摘要文件路径
            relative_path = os.path.relpath(file_path, directory_path)
            output_file_path = os.path.join(output_directory, f"{relative_path}_summary.txt")
            output_dir = os.path.dirname(output_file_path)
            os.makedirs(output_dir, exist_ok=True)  # 确保目录存在

            # 只写入摘要部分
            with open(output_file_path, 'w', encoding='utf-8') as summary_file:
                summary_file.write(f"{summary_content}\n")

            print(f"Summary for {filename} saved to {output_file_path}")

print("All summaries generated and saved.")

Summary for WebManager.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/WebManager.java_summary.txt
Summary for DataStorage.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/DataStorage.java_summary.txt
Summary for MainActivity.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/MainActivity.java_summary.txt
Summary for ValueProvider.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/ValueProvider.java_summary.txt
Summary for NumMessage.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/NumMessage.java_summary.txt
Summary for SecurityReceiver.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/SecurityReceiver.java_summary.txt
Summary for SecurityService.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/SecurityService.java_summary.txt
Summary for b.java saved to /content/drive/MyDrive/zitmo_banker_reversed/summaries/b.java_summary.txt
All summaries generated 